# Workshop dbt

En primer lugar descargamos e instalamos dbt (data build tool). En este caso usaremos la versión para duckdb, pero dbt soporta otros motores de bases de datos como PostgreSQL, Snowflake, BigQuery, etc.
Se recomienda crear un entorno virtual para instalar dbt y sus dependencias. En Windows podemos usar venv
Todo el software necesario se puede instalar con pip

Una vez instalado dbt y creado el entorno virtual, iniciamos un nuevo proyecto con el comando `dbt init` y seguimos las instrucciones en pantalla. Esto creará una estructura de directorios básica para nuestro proyecto dbt.
Así como un perfil en la carpeta Users/<user>/.dbt/profiles.yml

```markdown
(.venv) PS C:\GIT\dbt> dbt init
21:57:10  Running with dbt=1.10.9
Enter a name for your project (letters, digits, underscore): workshop_dbt
21:57:32  
Your new dbt project "workshop_dbt" was created!

For more information on how to configure the profiles.yml file,
please consult the dbt documentation here:

  https://docs.getdbt.com/docs/configure-your-profile

One more thing:

Need help? Don't hesitate to reach out to us via GitHub issues or on Slack:

  https://community.getdbt.com/

Happy modeling!

21:57:32  Setting up your profile.
Which database would you like to use?
[1] duckdb

(Don't see the one you want? https://docs.getdbt.com/docs/available-adapters)

Enter a number: 1
```

Para la conexión entre el proyecto dbt y la base de datos que se utilice se usarán los ficheros C:/Users/< user >/.dbt/profiles.yml y dbt_project.yml.

## Proyecto dbt

En este ejemplo vamos a ver la carga de una serie de ficheros en la capa bronze del proyecto para después transformarlos en las distintas capas mediante modelos dbt hasta llegar a un modelo analítico en la capa gold.

![arquitectura](arquitectura.svg)

In [2]:
cd workshop_dbt

c:\GIT\dbt\workshop_dbt


## Capa bronze

dbt es una herramienta de transformación de datos, no de ingesta de los mismos. Por tanto no aporta funcionalidades avanzadas para la carga de ficheros, sin embargo, sí que aporta la funcionalidad de seeds para la carga de datos para pequeñas tablas o para realizar validaciones de datos.

En primer lugar vamos a cargar las tablas en la capa bronze, para ello vamos a utilizar la propia funcionalidad de duckdb para carga de ficheros csv, en las tablas cabecera_tickets_manual, lineas_tickets_manual y centros_manual.

In [17]:
!duckdb.exe dev.db -s ".read analyses/tables_manual_bronze.sql"

┌───────────────┬─────────────────┬──────────────┐
│ total_centros │ total_cabeceras │ total_lineas │
│     int64     │      int64      │    int64     │
├───────────────┼─────────────────┼──────────────┤
│            10 │              20 │           82 │
└───────────────┴─────────────────┴──────────────┘
┌───────────────────────┬────────────┬────────────────────────┬───────────┬──────────────────────┬─────────┬─────────────────────────┐
│         tabla         │ cod_centro │     nombre_centro      │  ciudad   │        region        │ estado  │        ts_insert        │
│        varchar        │  varchar   │        varchar         │  varchar  │       varchar        │ varchar │        timestamp        │
├───────────────────────┼────────────┼────────────────────────┼───────────┼──────────────────────┼─────────┼─────────────────────────┤
│ bronze.centros_manual │ CENT001    │ Centro Comercial Norte │ Madrid    │ Madrid               │ Activo  │ 2025-08-30 17:51:11.735 │
│ bronze.centros_ma

Pasando a utilizar las funcionales de dbt también podemos cargar las tablas en la capa bronze con el comando: dbt seed, este comando buscará en todos los ficheros .yml el atributo "seeds" y lanzará la carga del fichero csv que tiene el mismo nombre siempre con la estrategia truncate/insert y sin dejar la opción de cargar distintos ficheros por cada tabla.

En el caso de nuestro proyecto tenemos un único fichero de seeds ubicado en la carpeta seeds que cargará las tablas cabecera_tickets, lineas_tickets y centros (las mismas que hemos cargado anteriormente eliminando el sufijo).

In [15]:
!dbt seed

15:48:52  Running with dbt=1.10.9
15:48:52  Registered adapter: duckdb=1.9.4
15:48:53  Found 14 models, 1 analysis, 3 seeds, 5 operations, 156 data tests, 6 sources, 849 macros
15:48:53  
15:48:53  Concurrency: 1 threads (target='dev')
15:48:53  
15:48:53  Iniciando ejecución de dbt
15:48:53  1 of 3 START hook: workshop_dbt.on-run-start.0 ................................. [RUN]
15:48:53  1 of 3 OK hook: workshop_dbt.on-run-start.0 .................................... [OK in 0.01s]
15:48:53  Tabla de auditoría ya existe: main.dbt_audit
15:48:53  2 of 3 START hook: workshop_dbt.on-run-start.1 ................................. [RUN]
15:48:53  2 of 3 OK hook: workshop_dbt.on-run-start.1 .................................... [OK in 0.01s]
15:48:53  Inicio de ejecución registrado en auditoría. ID: 9
15:48:53  3 of 3 START hook: workshop_dbt.on-run-start.2 ................................. [RUN]
15:48:53  3 of 3 OK hook: workshop_dbt.on-run-start.2 .................................... [OK in 0

In [18]:
!duckdb.exe dev.db -s ".read analyses/select_seed.sql"

┌───────────────┬─────────────────┬──────────────┐
│ total_centros │ total_cabeceras │ total_lineas │
│     int64     │      int64      │    int64     │
├───────────────┼─────────────────┼──────────────┤
│            10 │              20 │           82 │
└───────────────┴─────────────────┴──────────────┘
┌────────────────┬────────────┬────────────────────────┬───────────┬──────────────────────┬─────────┐
│     tabla      │ cod_centro │     nombre_centro      │  ciudad   │        region        │ estado  │
│    varchar     │  varchar   │        varchar         │  varchar  │       varchar        │ varchar │
├────────────────┼────────────┼────────────────────────┼───────────┼──────────────────────┼─────────┤
│ bronze.centros │ CENT001    │ Centro Comercial Norte │ Madrid    │ Madrid               │ Activo  │
│ bronze.centros │ CENT002    │ Centro Comercial Sur   │ Barcelona │ Cataluña             │ Activo  │
│ bronze.centros │ CENT003    │ Centro Comercial Este  │ Valencia  │ Comunidad Val

Sobre estos datos ya podemos pasar algunos tests que están definidos dentro del propio fichero seeds (también podían estar )

In [19]:
!dbt test --select tag:tag_seed

16:34:04  Running with dbt=1.10.9
16:34:05  Registered adapter: duckdb=1.9.4
16:34:05  Unable to do partial parsing because a project config has changed
16:34:06  Iniciando ejecución de dbt
16:34:06  Tabla de auditoría creada: main.dbt_audit
16:34:06  Inicio de ejecución registrado en auditoría. ID: 1
16:34:06  Finalizando ejecución de dbt
16:34:06  No se encontró ejecución pendiente para actualizar
16:34:06  Found 14 models, 2 analyses, 3 seeds, 5 operations, 156 data tests, 6 sources, 849 macros
16:34:06  
16:34:06  Concurrency: 1 threads (target='dev')
16:34:06  
16:34:07  Iniciando ejecución de dbt
16:34:07  1 of 3 START hook: workshop_dbt.on-run-start.0 ................................. [RUN]
16:34:07  1 of 3 OK hook: workshop_dbt.on-run-start.0 .................................... [OK in 0.01s]
16:34:07  Tabla de auditoría ya existe: main.dbt_audit
16:34:07  2 of 3 START hook: workshop_dbt.on-run-start.1 ................................. [RUN]
16:34:07  2 of 3 OK hook: workshop_d

In [20]:
!dbt test --select seeds/*

16:35:10  Running with dbt=1.10.9
16:35:10  Registered adapter: duckdb=1.9.4
16:35:11  Found 14 models, 2 analyses, 3 seeds, 5 operations, 156 data tests, 6 sources, 849 macros
16:35:11  
16:35:11  Concurrency: 1 threads (target='dev')
16:35:11  
16:35:11  Iniciando ejecución de dbt
16:35:11  1 of 3 START hook: workshop_dbt.on-run-start.0 ................................. [RUN]
16:35:11  1 of 3 OK hook: workshop_dbt.on-run-start.0 .................................... [OK in 0.01s]
16:35:11  Tabla de auditoría ya existe: main.dbt_audit
16:35:11  2 of 3 START hook: workshop_dbt.on-run-start.1 ................................. [RUN]
16:35:11  2 of 3 OK hook: workshop_dbt.on-run-start.1 .................................... [OK in 0.01s]
16:35:11  Inicio de ejecución registrado en auditoría. ID: 11
16:35:11  3 of 3 START hook: workshop_dbt.on-run-start.2 ................................. [RUN]
16:35:11  3 of 3 OK hook: workshop_dbt.on-run-start.2 .................................... [OK in 

Para la capa bronze definiremos también los ficheros source.yml para referenciar los datos que hemos cargado externamente en modelos posteriores. Además dbt permite comprobar si los origenes se han actualizado recientemente mediante el parámetro freshness.
Para realizar esta validación vamos a introducir también el concepto de variables de proyecto, estas estarán definidas en el fichero dbt_project.yml y podremos referenciarlas en los ficheros .yml mediante la función {{ var('nombre_variable') }}.

In [23]:
!dbt source freshness

16:58:45  Running with dbt=1.10.9
16:58:45  Registered adapter: duckdb=1.9.4
16:58:46  Found 14 models, 2 analyses, 3 seeds, 5 operations, 132 data tests, 3 sources, 849 macros
16:58:46  
16:58:46  Concurrency: 1 threads (target='dev')
16:58:46  
16:58:46  Iniciando ejecución de dbt
16:58:46  1 of 3 START hook: workshop_dbt.on-run-start.0 ................................. [RUN]
16:58:46  1 of 3 OK hook: workshop_dbt.on-run-start.0 .................................... [OK in 0.01s]
16:58:46  Tabla de auditoría ya existe: main.dbt_audit
16:58:46  2 of 3 START hook: workshop_dbt.on-run-start.1 ................................. [RUN]
16:58:46  2 of 3 OK hook: workshop_dbt.on-run-start.1 .................................... [OK in 0.01s]
16:58:46  Inicio de ejecución registrado en auditoría. ID: 12
16:58:46  3 of 3 START hook: workshop_dbt.on-run-start.2 ................................. [RUN]
16:58:46  3 of 3 OK hook: workshop_dbt.on-run-start.2 .................................... [OK in 

En este paso el proceso ha comprobado que los datos han sido cargados por debajo del valor de error de fresheness, en caso contrario el proceso habría fallado.
Como podemos comprobar lanzando el proceso una hora más tarde.

In [3]:
!dbt source freshness

17:08:00  Running with dbt=1.10.9
17:08:00  Registered adapter: duckdb=1.9.4
17:08:02  Found 14 models, 2 analyses, 3 seeds, 5 operations, 132 data tests, 3 sources, 849 macros
17:08:02  
17:08:02  Concurrency: 1 threads (target='dev')
17:08:02  
17:08:03  Iniciando ejecución de dbt
17:08:03  1 of 3 START hook: workshop_dbt.on-run-start.0 ................................. [RUN]
17:08:03  1 of 3 OK hook: workshop_dbt.on-run-start.0 .................................... [OK in 0.01s]
17:08:03  Tabla de auditoría ya existe: main.dbt_audit
17:08:03  2 of 3 START hook: workshop_dbt.on-run-start.1 ................................. [RUN]
17:08:03  2 of 3 OK hook: workshop_dbt.on-run-start.1 .................................... [OK in 0.01s]
17:08:03  Inicio de ejecución registrado en auditoría. ID: 13
17:08:03  3 of 3 START hook: workshop_dbt.on-run-start.2 ................................. [RUN]
17:08:03  3 of 3 OK hook: workshop_dbt.on-run-start.2 .................................... [OK in 

## Capa silver

Para la capa silver vamos a consolidar la información probando distintas estrategias de materialización existentes en dbt: truncate (table), merge (materialize con n unique_key) y append (materialize).
Posteriormente utilizaremos los modelos merge para consolidar la información mediante los modelos _consolidados

En los modelos de la capa silver se utilizan variables pasadas por línea de comandos para definir el rango de fechas de los datos a procesar.
Para ello utilizaremos el siguiente comando a la hora de ejecutar los modelos:
 * dbt run --select tag:tag_consolidados --vars '{"start_date": "2024-01-01", "end_date": "2024-01-31"}'
 * dbt test --select tag:tag_consolidados --vars '{"start_date": "2024-01-01", "end_date": "2024-01-31"}'
 * dbt build --select tag:tag_consolidados --vars '{"start_date": "2024-01-01", "end_date": "2024-01-31"}'

In [15]:
!dbt run --select tag:tag_consolidados --vars "{\"start_date\": \"2024-01-01\", \"end_date\": \"2024-01-31\"}"

17:20:27  Running with dbt=1.10.9
17:20:27  Registered adapter: duckdb=1.9.4
17:20:27  Found 14 models, 2 analyses, 3 seeds, 5 operations, 132 data tests, 3 sources, 849 macros
17:20:27  
17:20:27  Concurrency: 1 threads (target='dev')
17:20:27  
17:20:27  Iniciando ejecución de dbt
17:20:27  1 of 3 START hook: workshop_dbt.on-run-start.0 ................................. [RUN]
17:20:27  1 of 3 OK hook: workshop_dbt.on-run-start.0 .................................... [OK in 0.01s]
17:20:27  Tabla de auditoría ya existe: main.dbt_audit
17:20:27  2 of 3 START hook: workshop_dbt.on-run-start.1 ................................. [RUN]
17:20:27  2 of 3 OK hook: workshop_dbt.on-run-start.1 .................................... [OK in 0.01s]
17:20:27  Inicio de ejecución registrado en auditoría. ID: 16
17:20:27  3 of 3 START hook: workshop_dbt.on-run-start.2 ................................. [RUN]
17:20:27  3 of 3 OK hook: workshop_dbt.on-run-start.2 .................................... [OK in 

In [17]:
!dbt test --select tag:tag_consolidados --vars "{\"start_date\": \"2024-01-01\", \"end_date\": \"2024-01-31\"}"

17:21:02  Running with dbt=1.10.9
17:21:02  Registered adapter: duckdb=1.9.4
17:21:02  Found 14 models, 2 analyses, 3 seeds, 5 operations, 132 data tests, 3 sources, 849 macros
17:21:02  
17:21:02  Concurrency: 1 threads (target='dev')
17:21:02  
17:21:02  Iniciando ejecución de dbt
17:21:02  1 of 3 START hook: workshop_dbt.on-run-start.0 ................................. [RUN]
17:21:02  1 of 3 OK hook: workshop_dbt.on-run-start.0 .................................... [OK in 0.01s]
17:21:02  Tabla de auditoría ya existe: main.dbt_audit
17:21:02  2 of 3 START hook: workshop_dbt.on-run-start.1 ................................. [RUN]
17:21:02  2 of 3 OK hook: workshop_dbt.on-run-start.1 .................................... [OK in 0.01s]
17:21:03  Inicio de ejecución registrado en auditoría. ID: 18
17:21:03  3 of 3 START hook: workshop_dbt.on-run-start.2 ................................. [RUN]
17:21:03  3 of 3 OK hook: workshop_dbt.on-run-start.2 .................................... [OK in 

In [16]:
!dbt build --select tag:tag_consolidados --vars "{\"start_date\": \"2024-01-01\", \"end_date\": \"2024-01-31\"}"

17:20:43  Running with dbt=1.10.9
17:20:43  Registered adapter: duckdb=1.9.4
17:20:44  Found 14 models, 2 analyses, 3 seeds, 5 operations, 132 data tests, 3 sources, 849 macros
17:20:44  
17:20:44  Concurrency: 1 threads (target='dev')
17:20:44  
17:20:44  Iniciando ejecución de dbt
17:20:44  1 of 3 START hook: workshop_dbt.on-run-start.0 ................................. [RUN]
17:20:44  1 of 3 OK hook: workshop_dbt.on-run-start.0 .................................... [OK in 0.01s]
17:20:44  Tabla de auditoría ya existe: main.dbt_audit
17:20:44  2 of 3 START hook: workshop_dbt.on-run-start.1 ................................. [RUN]
17:20:44  2 of 3 OK hook: workshop_dbt.on-run-start.1 .................................... [OK in 0.01s]
17:20:44  Inicio de ejecución registrado en auditoría. ID: 17
17:20:44  3 of 3 START hook: workshop_dbt.on-run-start.2 ................................. [RUN]
17:20:44  3 of 3 OK hook: workshop_dbt.on-run-start.2 .................................... [OK in 

Si vamos a la carpeta target podemos ver como se han implementado los diferentes modelos, en este caso estamos utilizando distintas estrategias de materialización dentro de append (los distintos tipos de materialización, así como las estretegias que pueden utilizarse difieren en función de la base de datos) )

Los tests de estos modelos se han definido aprovechando el propio yaml del source, en ellos tenemos tests propios de dbt (not_null, unique, relationships), tests singulares (personalizados para validar que en un descriptivo existe al menos una mayúscula) y tests descargados de dbt-utils (unique de varios valores).

## Capa gold

La capa gold es la capa final en un pipeline de datos, donde los datos han sido completamente transformados, limpiados y enriquecidos para su uso en análisis y toma de decisiones. Esta capa está pensada para ser utilizada por los usuarios finales, ya sea mediante queries de autoservicio como dashboards o informes.

Hemos aprovechado esta capa para introducir la ejecución de queries antes (o después de la ejecución de los modelos dbt) mediante los atributos pre_hook y post_hook.

In [23]:
!dbt build --select carpeta_gold* --vars "{\"start_date\": \"2024-01-01\", \"end_date\": \"2024-01-31\"}"

17:48:54  Running with dbt=1.10.9
17:48:54  Registered adapter: duckdb=1.9.4
17:48:54  Found 14 models, 2 analyses, 3 seeds, 5 operations, 132 data tests, 3 sources, 849 macros
17:48:54  
17:48:54  Concurrency: 1 threads (target='dev')
17:48:54  
17:48:55  Iniciando ejecución de dbt
17:48:55  1 of 3 START hook: workshop_dbt.on-run-start.0 ................................. [RUN]
17:48:55  1 of 3 OK hook: workshop_dbt.on-run-start.0 .................................... [OK in 0.01s]
17:48:55  Tabla de auditoría ya existe: main.dbt_audit
17:48:55  2 of 3 START hook: workshop_dbt.on-run-start.1 ................................. [RUN]
17:48:55  2 of 3 OK hook: workshop_dbt.on-run-start.1 .................................... [OK in 0.01s]
17:48:55  Inicio de ejecución registrado en auditoría. ID: 19
17:48:55  3 of 3 START hook: workshop_dbt.on-run-start.2 ................................. [RUN]
17:48:55  3 of 3 OK hook: workshop_dbt.on-run-start.2 .................................... [OK in 

## Monitorización y linage

Para monitorizar la ejecución de los diferentes modelos, tests, seeds y, en general, cualquier comando dbt se pueden utilizar los artefactos, sin entrar en detalles, los artefactos son una serie de ficheros JSON que se generan en la carpeta target después de ejecutar cualquier comando dbt. Estos ficheros contienen información muy útil sobre la ejecución, como por ejemplo:
- El estado de la ejecución (éxito, fallo, advertencia, etc.)
- El tiempo que ha tardado en ejecutarse cada modelo o test.
- Los errores que se han producido durante la ejecución.

Estos objetos pueden ser referenciados mediante variables internas de dbt, por ejemplo:
- `{{ run_started_at }}`: Fecha y hora de inicio de la ejecución.
- `{{ run_finished_at }}`: Fecha y hora de finalización de la ejecución.
- `{{ run_duration }}`: Duración total de la ejecución en segundos.
- `{{ invocation_id }}`: Identificador único de la ejecución.
...

Vamos a aprovechar la existencia de dichas variables para crear un modelo que nos permita monitorizar las ejecuciones de los diferentes modelos y tests.
En este caso configuraremos el modelo dbt_audit para ello.

In [ ]:
!duckdb.exe dev.db -s ".read analyses/select_audit.sql"